In [2]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from numba import njit
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import random

In [3]:
def calculate_epsilon(T):
    epsilon_values = []
    
    for i in range(T):
        theta = -((1/1000000) ** (1/T)) + 1
        epsilon = (1 - theta) ** i
        epsilon_values.append(epsilon)
    
    return epsilon_values

In [4]:
def demand(p1t,p2t):
    """
    args:
        p1t: price of agent 1
        p2t: price of agent 2
    returns:
        d: demand for given set of prices
    """
    if p1t<p2t:
        d = 1-p1t
    elif p1t==p2t:
        d = 0.5*(1-p1t)
    else:
        d = 0
    return d

In [33]:
def profit(p1t, p2t, prices):
    """
    args:
        p1t: index price of agent 1
        p2t: index price of agent 2
        prices: vector of prices
    returns:
        profit for agent
    """
    return prices[p1t] * demand(prices[p1t], prices[p2t])
 

In [34]:
def select_price_greedy(Q, current_s, p, epsilon):
    """Epsilon-greedy action selection.
    args:
        Q: Q-function
        current_s: current state
        p: price vector containing the possible prices
        epsilon: probability of selecting an action uniformly at random
    returns:
        the index of selected action
    """
    u = random.uniform(0,1)
    if u < epsilon:
        #return np.random.choice(p)
        random_index = np.random.choice(len(p))
        return random_index
    else:

        max_idx = np.argmax(Q[np.where(p == current_s)[0][0], :])
        #max_idx = np.argmax(Q[current_s, :])
        return max_idx

In [73]:
def Q_func(price_idx, current_state_idx, q_table, price_grid, delta, epsilon, alpha, it):
    """
    args
        price_idx: current price index
        current_state_idx: current state index
        q_table: q_table for 1 player
        price_grid: price grid
        delta: discount factor
        epsilon: probability of exploration
        alpha: step-size parameter
    returns:
        updated Q-table
    """
    #print("price_idx: ", price_idx)
    #print("current_state_idx", current_state_idx)
    prev_est = q_table[price_idx, current_state_idx]
    next_state_idx = select_price_greedy(q_table, price_grid[current_state_idx], price_grid, epsilon[it])
    new_est = profit(price_idx, current_state_idx, price_grid) + delta * profit(price_idx, next_state_idx, price_grid) + delta**2 * np.argmax(q_table[:, next_state_idx])
    q_table[price_idx, current_state_idx] = (1 - alpha) * prev_est + alpha * new_est
    return next_state_idx
    

In [111]:
def simulation(alpha, delta, T, price_grid):
    epsilon = calculate_epsilon(T)
    k = len(price_grid)
    q1 = np.zeros((k, k))
    q2 = np.zeros((k, k))

    profits = np.zeros((2, T))
    avg_prof1 = []
    avg_prof2 = []

    i = 0
    j = 1
    t = 0

    p_table = np.zeros((2, T))

    p_table[i,t] = int(np.random.choice(len(price_grid)))
    t += 1
    p_table[j,t] = int(np.random.choice(len(price_grid)))
    p_table[i,t] = p_table[i,t-1]

    t += 1

    for t in range(t, T - 1):
        if i == 0:
            #print(p_table)
            #print(t)
            #print(p_table[i,t -2])
            p_table[i,t] = Q_func(int(p_table[i, t-2]), int(p_table[j, t-2]), q1, price_grid, delta, epsilon, alpha, t)
            p_table[j, t] = p_table[j, t-1]
            profits[i, t] = profit(int(p_table[i,t]), int(p_table[j,t]), price_grid)
            if t % 1000 == 0:
                profitability = np.sum(profits[i, (t-1000):1000])/1000
                avg_prof1.append(profitability)
        else:
            p_table[i,t] = Q_func(int(p_table[i, t-2]), int(p_table[j, t-2]), q2, price_grid, delta, epsilon, alpha, t)
            p_table[j, t] = p_table[j, t-1]
            profits[j, t] = profit(int(p_table[i,t]), int(p_table[j,t]), price_grid)
            if t % 1000 == 0:
                profitability = np.sum(profits[j, (t-1000):1000])/1000
                avg_prof2.append(profitability)
        tmp = i   
        i = j
        j = tmp
    return p_table, q1, q2, profits, avg_prof1, avg_prof2




In [112]:
a,b,c,d,e,f =simulation(0.3, 0.95, 5000000, np.linspace(1/6, 1, 6))
print(a[0,:10])
print(a[1,:10])
print(b)


[5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[0. 3. 3. 3. 3. 0. 0. 3. 3. 1.]
[[3.20465619e+00 3.68580462e+00 1.17333333e+00 2.87602957e+00
  3.46405821e+00 2.61769299e+00]
 [3.71555556e+00 2.34534117e+00 3.01154043e+00 3.14083333e+00
  3.51813274e+00 3.41959521e+00]
 [1.84110002e+00 2.94499981e+00 1.26500000e+00 3.07002099e+00
  1.39025417e+00 2.83399881e+00]
 [3.46432348e+00 2.11111226e-01 1.91055555e+00 3.02923569e+00
  3.81855837e+00 3.54698433e+00]
 [3.61000000e+00 3.76527106e+00 2.13887358e+00 2.77347222e+00
  3.77316687e+00 3.84249807e+00]
 [3.58551615e+00 9.02493908e-01 2.34209015e-06 2.70150146e+00
  2.79775000e+00 3.60992375e+00]]


In [113]:
print(d[0,:10])
print(d[1,:10])
#print(e)
#print(f)

[0.         0.         0.         0.22222222 0.         0.13888889
 0.         0.22222222 0.         0.22222222]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
a = [4, 5]
np.random.choice(a)

5